## **Installing pre-reqs**

In [2]:
!pip -q install faiss-cpu sentence-transformers langchain langchain-community langchain-text-splitters pypdf python-docx llama-cpp-python pandas tqdm requests

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.7/50.7 MB 19.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 73.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 82.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.5/310.5 kB 28.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.0/253.0 kB 25.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.8 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1

## **Setting up google drive and folders**

In [3]:
from google.colab import drive
drive.mount('/content/drive')  # uncomment if you want persistence

import os, pathlib
BASE_DIR = "/content/drive/MyDrive/Colab"
DATA_DIR = f"{BASE_DIR}/dataset_raw"   # raw zips/files
DOCS_DIR = f"{BASE_DIR}/docs"          # normalized text
OUT_DIR  = f"{BASE_DIR}/rag_outputs"   # indices, exports, model
for d in (DATA_DIR, DOCS_DIR, OUT_DIR): pathlib.Path(d).mkdir(parents=True, exist_ok=True)

Mounted at /content/drive


## **Getting data from zenodo**

In [4]:
import requests, os, json, re, shutil, zipfile
from pathlib import Path

ZENODO_REC_ID = "13880060"  # Requirements data sets (user stories)
api_url = f"https://zenodo.org/api/records/{ZENODO_REC_ID}"
rec = requests.get(api_url, timeout=60).json()

files = rec.get("files", []) or rec.get("metadata", {}).get("related_identifiers", [])  # fallback
downloaded = []

if "files" in rec:
    for f in rec["files"]:
        url = f["links"]["self"]
        name = f["key"]
        dest = Path(DATA_DIR)/name
        if not dest.exists():
            with requests.get(url, stream=True, timeout=120) as r:
                r.raise_for_status()
                with open(dest, "wb") as fh:
                    shutil.copyfileobj(r.raw, fh)
        downloaded.append(str(dest))
else:
    print("Could not find file list in record JSON; open the Zenodo page and download manually.")

print(f"Downloaded {len(downloaded)} files")
downloaded[:8]

Downloaded 22 files


['/content/drive/MyDrive/Colab/dataset_raw/g16-mis.txt',
 '/content/drive/MyDrive/Colab/dataset_raw/g21-badcamp.txt',
 '/content/drive/MyDrive/Colab/dataset_raw/g17-cask.txt',
 '/content/drive/MyDrive/Colab/dataset_raw/g10-scrumalliance.txt',
 '/content/drive/MyDrive/Colab/dataset_raw/g26-racdam.txt',
 '/content/drive/MyDrive/Colab/dataset_raw/g13-planningpoker.txt',
 '/content/drive/MyDrive/Colab/dataset_raw/g12-camperplus.txt',
 '/content/drive/MyDrive/Colab/dataset_raw/g19-alfred.txt']

In [5]:
for p in list(Path(DATA_DIR).glob("*.zip")):
    with zipfile.ZipFile(p, 'r') as zf:
        zf.extractall(DATA_DIR)
    print("Extracted:", p.name)

## **Normalize everything to plain text**

In [6]:
import re, glob
from pypdf import PdfReader
from docx import Document

def file_to_text(path):
    p = path.lower()
    if p.endswith(".pdf"):
        try:
            pages = []
            reader = PdfReader(path)
            for i, pg in enumerate(reader.pages, start=1):
                t = pg.extract_text() or ""
                pages.append(f"[PAGE {i}] {t}")
            return "\n".join(pages)
        except: return ""
    if p.endswith(".docx"):
        try:
            doc = Document(path)
            return "\n".join(p.text for p in doc.paragraphs)
        except: return ""
    if any(p.endswith(ext) for ext in (".txt",".md",".rst",".csv")):
        try:
            with open(path, "r", errors="ignore") as f: return f.read()
        except: return ""
    return ""  # unsupported

def clean_text(s):
    s = s.replace("\x00"," ")
    s = re.sub(r"[ \t]+"," ", s)
    s = re.sub(r"\n{3,}","\n\n", s)
    return s.strip()

import os, pathlib
count=0
for root, _, files in os.walk(DATA_DIR):
    for fn in files:
        src = os.path.join(root, fn)
        txt = clean_text(file_to_text(src))
        if txt and len(txt) > 50:
            out = f"{DOCS_DIR}/{pathlib.Path(fn).stem}.txt"
            with open(out, "w") as f: f.write(txt)
            count += 1
print("Normalized text files:", count)

Normalized text files: 22


## **Chunking, Embedding, FAISS**

In [7]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from sentence_transformers import SentenceTransformer
import numpy as np, faiss, pickle, glob, os

# Load normalized docs
docs = []
for fp in sorted(glob.glob(f"{DOCS_DIR}/*.txt")):
    with open(fp, "r", errors="ignore") as f:
        docs.append({"source": os.path.basename(fp), "text": f.read()})
print("Docs:", len(docs))

# Chunk
splitter = RecursiveCharacterTextSplitter(
    chunk_size=1200, chunk_overlap=150,
    separators=["\n## ","\n# ","\n\n","\n"," ",""]
)
chunks = []
for d in docs:
    for ch in splitter.split_text(d["text"]):
        chunks.append({"source": d["source"], "text": ch})
print("Chunks:", len(chunks))

# Embeddings + FAISS (cosine via normalized IP)
EMB_NAME = "sentence-transformers/all-MiniLM-L6-v2"
embed_model = SentenceTransformer(EMB_NAME)

B=128; vecs=[]
texts = [c["text"] for c in chunks]
for i in range(0, len(texts), B):
    vecs.append(embed_model.encode(texts[i:i+B], convert_to_numpy=True, normalize_embeddings=True))
vecs = np.vstack(vecs).astype("float32")

index = faiss.IndexFlatIP(vecs.shape[1])
index.add(vecs)

# Persist
with open(f"{OUT_DIR}/chunks.pkl","wb") as f: pickle.dump(chunks,f)
faiss.write_index(index, f"{OUT_DIR}/faiss.index")
with open(f"{OUT_DIR}/emb_model.txt","w") as f: f.write(EMB_NAME)
print("Index built.")

Docs: 22
Chunks: 42


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Index built.


**Retriever helper**

In [8]:
def retrieve(query, k=8):
    q = embed_model.encode([query], convert_to_numpy=True, normalize_embeddings=True).astype("float32")
    D, I = index.search(q, k)
    hits=[]
    for rank,(idx,score) in enumerate(zip(I[0],D[0]), start=1):
        hits.append({"rank":rank,"score":float(score),"source":chunks[idx]["source"],"text":chunks[idx]["text"]})
    return hits

## **Local LLM via llama_cpp**


In [9]:
import os, subprocess, pathlib

MODEL_URL = "https://huggingface.co/TheBloke/Mistral-7B-Instruct-v0.2-GGUF/resolve/main/mistral-7b-instruct-v0.2.Q4_K_M.gguf"
MODEL_PATH = f"{OUT_DIR}/mistral-7b-instruct.Q4_K_M.gguf"
if not os.path.exists(MODEL_PATH):
    !wget -q -O "$MODEL_PATH" "$MODEL_URL"

from llama_cpp import Llama
llm = Llama(model_path=MODEL_PATH, n_ctx=8192, n_threads=8, n_gpu_layers=0, temperature=0.2)

def chat_local(messages, max_tokens=1024, temperature=0.2):
    out = llm.create_chat_completion(messages=messages, temperature=temperature, max_tokens=max_tokens)
    return out["choices"][0]["message"]["content"]

llama_model_loader: loaded meta data with 24 key-value pairs and 291 tensors from /content/drive/MyDrive/Colab/rag_outputs/mistral-7b-instruct.Q4_K_M.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = mistralai_mistral-7b-instruct-v0.2
llama_model_loader: - kv   2:                       llama.context_length u32              = 32768
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 32
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 14336
llama_model_loader: - kv   6:                 llama.rope.dimension_count u32              = 128
llama_model

## **Prompt and RAG generation**

In [10]:
TESTCASE_SYSTEM = """You are a senior QA engineer. Generate rigorous, unambiguous test cases
based ONLY on the provided context snippets. If context is missing details, list assumptions and gaps."""

TESTCASE_USER_TEMPLATE = """Context (RAG snippets):
---
{context}
---
Task: Generate test cases for the scope: "{scope}".
If acceptance criteria (AC) appear in the context, map each test to at least one AC id/phrase.

Output (Markdown):
- Feature: <name>
- Scope: <scope>
- Assumptions/Notes: <gaps or clarifications needed>
- Test Cases (table):
  | ID | Title | Type (pos/neg/edge) | Pre-Conditions | Steps | Expected Result | Priority (H/M/L) | MapsTo (AC id/phrase) |
After the table:
- Coverage: bullets of ACs covered and any missing ACs
- Additional Negative/Edge Ideas: bullets
"""

def make_context(hits, max_chars=1000):
    blocks=[]
    for h in hits:
        t = h["text"].strip()
        if len(t) > max_chars: t = t[:max_chars] + "..."
        blocks.append(f"[{h['source']} • score={h['score']:.2f}]\n{t}")
    return "\n\n".join(blocks)

def generate_test_cases(scope_query, k=8, use="local"):
    hits = retrieve(scope_query, k=k)
    ctx = make_context(hits)
    messages = [
        {"role":"system","content":TESTCASE_SYSTEM},
        {"role":"user","content":TESTCASE_USER_TEMPLATE.format(context=ctx, scope=scope_query)}
    ]
    out = chat_local(messages)  # or chat_api(messages)
    return out, hits

In [11]:
#trying it
md_text, evidence = generate_test_cases(
    "User registration: email verification, strong password policy, duplicate email handling",
    k=10, use="local"
)
print(md_text[:1500])

llama_perf_context_print:        load time = 1966329.00 ms
llama_perf_context_print: prompt eval time = 1966328.12 ms /  7994 tokens (  245.98 ms per token,     4.07 tokens per second)
llama_perf_context_print:        eval time =  164831.58 ms /   197 runs   (  836.71 ms per token,     1.20 tokens per second)
llama_perf_context_print:       total time = 2131282.87 ms /  8191 tokens
llama_perf_context_print:    graphs reused =        190


 ## Test Cases for User Registration: Email Verification, Strong Password Policy, Duplicate Email Handling ##

| ID | Title | Type (Pos/Neg/Edge) | Pre-Conditions | Steps | Expected Result | Priority (H/M/L) | MapsTo (AC id/phrase) |
| --- | --- | --- | --- | --- | --- | --- | --- |
| 1 | Valid Email Verification | Positive | New user, valid email format | Registers with valid email | Email is verified and account created | High | Email Verification |
| 2 | Invalid Email Verification | Negative | New user, invalid email format | Registration fails with error message | Email is not verified and account not created | High | Email Verification |
| 3 | Duplicate Email Handling | Negative | New user, existing email in system | Registration fails with error
